In [44]:
import pandas as pd
from pathlib import Path
import os
import pinyin
import translators
from xml.etree import ElementTree
import datetime, time
import numpy as np

import re

In [2]:
PathForNotes = Path('/home/jentlejames/Downloads') 
main_vocab_df = pd.read_csv('./vocab_master.csv',index_col=0)

# Target for automation
#fileName = 'WF - Notes - 220722-110837.opml'

In [142]:

def find_most_recent_ompl(base_dir):
    opmlFileList = []

    # fetches all opml files

    for file in os.listdir(base_dir):
        if file.endswith('.opml'):
            opmlFileList.append(file)

    # Sorts them by timestamp, get the index of most recent file

    timestampPattern = re.compile(r'[0-9]{6}\-[0-9]{6}')

    fileTimestamps = []
    stringToDatetimePattern = "%y%m%d-%H%M%S"

    for file in opmlFileList:
        timestamp = re.search(timestampPattern, file)[0]
        timestamp = time.mktime(datetime.datetime.strptime(timestamp, stringToDatetimePattern).timetuple())
        fileTimestamps.append(timestamp)

    mostRecent = max(fileTimestamps)
    mostRecentIdx = fileTimestamps.index(mostRecent)

    return base_dir/opmlFileList[mostRecentIdx]


# Chinese Japanese Character Ranges
# Functions

def is_cjk(char):
    char = ord(char)
    cjk_ranges = [
    (0x4E00,  0x62FF),
    (0x6300,  0x77FF),
    (0x7800,  0x8CFF),
    (0x8D00,  0x9FCC),
    (0x3400,  0x4DB5),
    (0x20000, 0x215FF),
    (0x21600, 0x230FF),
    (0x23100, 0x245FF),
    (0x24600, 0x260FF),
    (0x26100, 0x275FF),
    (0x27600, 0x290FF),
    (0x29100, 0x2A6DF),
    (0x2A700, 0x2B734),
    (0x2B740, 0x2B81D),
    (0x2B820, 0x2CEAF),
    (0x2CEB0, 0x2EBEF),
    (0x2F800, 0x2FA1F), ]
    
    
    for bottom, top in cjk_ranges:
        if char >= bottom and char <= top:
            return True
    return False

# Adds new words
def new_vocab_df_generator(elem):
    """_summary_
    
    Takes in an xml element, returns a dataframe of the chinese characters in each element
    along with the date
    
    Added
    
    Args:
        elem (_type_): _description_
        xml element, expected to be in the opml format from workflowy

    Returns:
        _type_: _description_
    """
    
    #vocab_df = pd.DataFrame({}, columns=['Characters', 'Date'])

    vocabFromCat = []
    catName = elem.attrib['text']
    for lessonNotes in elem.iter():
        # print(elementsDateAdded[i])
        
        for note in lessonNotes:
            #print(note)
            noteText = note.attrib['text']

            chineseCharacters = ''

            for char in noteText:
                if is_cjk(char):
                    chineseCharacters += char

            vocabFromCat.append(chineseCharacters)

        # Generates a column of same shape as vocab
        #dateColumn = [elementsDateAdded[i]] * len(vocabFromLesson)
        
        #lessonDictionary = dict(zipvocabFromLesson)
        catCol = [catName] * len(vocabFromCat)
        #catDictionary = dict(zip(vocabFromCat,catCol))
        tmp_df = pd.DataFrame({'Characters':vocabFromCat,
                               'Cat':catCol},dtype='object')
        print(tmp_df.shape)
        return tmp_df


In [6]:

######### BEGIN PROGRAM ############

fileName = find_most_recent_ompl(PathForNotes)
print(fileName)

/home/jentlejames/Downloads/WF - Export - 220725-132311.opml


In [22]:
    #sys.argv[1]


with open(fileName, 'rt') as f:
        tree = ElementTree.parse(f)

notesRoot = tree.getroot()

notesBody = notesRoot.find('body')
#outlineRoot = notesBody.find('outline')

In [ ]:

chineseBaseElem = None
for elem in notesBody:
    #print(elem.attrib['text'])
    if elem.attrib['text'] == 'Focus Points':
        chineseBaseElem = elem

for elem in chineseBaseElem:
    print(elem.attrib['text'])
    if elem.attrib['text'] == 'Chinese':
        chineseBaseElem = elem

for elem in chineseBaseElem:
    print(elem.text)
    if elem.attrib['text'] == '好好学习，天天上上':
        chineseBaseElem = elem



@unpackAI @ryzomatic #daily #easywin #elan
To Do Matrix
Inbox
Life Aspirations

Schedule
Focus Points
other
Inbox
Daily Routines
UnpackAI
 Elan #elan
Chinese
ORCA
Notes
On Deck
Matrix


In [24]:
vocabCategoriesBaseElem = None
tutorSessionNotesBaseElem = None

for elem in chineseBaseElem:
    #print(elem.attrib['text'])
    if elem.attrib['text'] == 'Categories':
        vocabCategoriesBaseElem = elem
    elif elem.attrib['text'] == 'Tutor Session Notes':
        tutorSessionNotesBaseElem = elem
    else:
        continue
        

Study Plans
videos 
Categories
Inbox
Tutor Session Notes
社交平台 she4 jiao1 ping2 tai2


In [25]:
tutorSessionNotesBaseElem

<Element 'outline' at 0x70831c337220>

In [109]:
vocabCats = []
categoryPattern = re.compile('Cat\s[0-9]*')

for elem in vocabCategoriesBaseElem:
    
    if re.search(categoryPattern, elem.attrib['text'][:5]):
        vocabCats.append(elem)

    

In [100]:
vocabCats[0].attrib['text']

'Cat 1 (Tech & Electronics) Work'

In [143]:
cat_df = pd.DataFrame({},columns=['Characters','Cat'])

for category in vocabCats:
    tmp_df = new_vocab_df_generator(category)
    
    cat_df = pd.concat([cat_df,tmp_df],ignore_index=True)

(63, 2)
(88, 2)
(103, 2)
(102, 2)
(0, 2)


In [145]:
cat_df['Pinyin'] = cat_df['Characters'].apply(lambda letters: pinyin.get(letters))

# Calls Google Translate API to translate new vocab
definitions = []
for term in cat_df['Characters'].iteritems():
    # Slow calls 
    definition = translators.google(term[1],from_language='zh-CN',to_language='en')
    definitions.append(definition)
cat_df['Meaning'] = definitions

In [147]:
cat_df.to_csv('categories_main.csv')

In [27]:

# Looks for element text which matches internal 
# workflowy pattern in opml 
timePattern = re.compile('time startYear')

# Creating list of elements
# Which contain lesson notes 
elementsByDate = []
elementsDateAdded = []


# Checks for elements which contain dates
# Which will contain sub elements with vocab
# Appends them to a list, along with the dates
# From before
for outline in tutorSessionNotesBaseElem.iter():

    text = outline.attrib['text']
    
    #Extracts the Date 
    # finds new nodes
    
    hasTime = re.search(timePattern,text)
    
    if hasTime:
        #Extracting datetime info
        time = text.split()
        year = time[1].split('\"')[1]
        month = time[2].split('\"')[1]
        day = time[3].split('\"')[1]
        dateAdded = year+' '+month +' '+day
        #print(dateAdded)
        
        elementsByDate.append(outline)
        elementsDateAdded.append(dateAdded)
    else:
        continue 

#print(elementsByDate)


In [32]:
elementsByDate

[<Element 'outline' at 0x70831c337270>,
 <Element 'outline' at 0x70831c33c090>,
 <Element 'outline' at 0x70831c33c680>,
 <Element 'outline' at 0x70831c2c51d0>,
 <Element 'outline' at 0x70831c2c5ae0>,
 <Element 'outline' at 0x70831c2d34f0>,
 <Element 'outline' at 0x70831c2d3ef0>,
 <Element 'outline' at 0x70831c2d8950>,
 <Element 'outline' at 0x70831c2d8bd0>,
 <Element 'outline' at 0x70831c2d8f40>,
 <Element 'outline' at 0x70831c2e5b80>,
 <Element 'outline' at 0x70831c2ebef0>,
 <Element 'outline' at 0x70831c2ef630>,
 <Element 'outline' at 0x70831c2efe50>,
 <Element 'outline' at 0x70831c28c450>]

In [35]:
tutorSessionDateSeries = pd.to_datetime(elementsDateAdded)

In [39]:
main_vocab_df[main_vocab_df['Date'].isin(tutorSessionDateSeries) == True]

,Characters,Date,Pinyin,Meaning,cluster


In [57]:
#newSessions = pd.Series(tutorSessionDateSeries[~tutorSessionDateSeries.isin(main_vocab_df.Date.unique())])
# Fill this up automatically


In [62]:
#for i, sessionDate in newSessions.iteritems():
#    print(sessionDate.date())
    

2022-06-08
2022-06-01


In [ ]:


    # dict(zip(vocabFromLesson,dateColumn))

    df_dictionary = pd.DataFrame(lessonDictionary, columns=['Characters', 'Date'])

    # print(df_dictionary.head())
    vocab_df = pd.concat([vocab_df, df_dictionary], ignore_index=True)


In [65]:

# Adds new words
def new_vocab_df_generator(elem):
    """_summary_
    
    Takes in an xml element, returns a dataframe of the chinese characters in each element
    along with the date
    
    Added
    
    Args:
        elem (_type_): _description_
        xml element, expected to be in the opml format from workflowy

    Returns:
        _type_: _description_
    """
    
    vocab_df = pd.DataFrame({}, columns=['Characters', 'Date'])

    for i, lessonNotes in new_sessions.iter:

        vocabFromLesson = []

        # print(elementsDateAdded[i])
        for note in lessonNotes:
            noteText = note.attrib['text']

            chineseCharacters = ''

            for char in noteText:
                if is_cjk(char):
                    chineseCharacters += char

            vocabFromLesson.append(chineseCharacters)

        # Generates a column of same shape as vocab
        dateColumn = [elementsDateAdded[i]] * len(vocabFromLesson)
        lessonDictionary = {'Characters': vocabFromLesson,
                            'Date': dateColumn}
        # dict(zip(vocabFromLesson,dateColumn))

        df_dictionary = pd.DataFrame(lessonDictionary, columns=['Characters', 'Date'])

        return df_dictionary




In [43]:
np.where(vocab_df['Date'].isin(tutorSessionDateSeries) == False)

NameError: name 'np' is not defined